# Requests

`requests` is a Python library that allows you to send http requests. Read more about this library in the [official documentation](https://requests.readthedocs.io/en/latest/#).

In the following cell we create container of the `httpbin` which allows to check headers of the requests.

In [3]:
import docker
import requests

docker_client = docker.from_env()

from src.rerun_docker import reload_docker_container
reload_docker_container(
    name="httpbin",
    image="kennethreitz/httpbin",
    ports={80: 80},
    detach=True,
    remove=True
)

<Container: db32b570689a>

## Default headers

There are some headers that the `requests` library generates itself - even if you've specified empty headers, there will be some headers in the request.

---

The following example shows that requests with headers specified as empty lead to requests that still have some headers.

By sending the request to the `httpbin` we created earlier, in the `/headers` path, we can check the headers of the request - they'll be returned as a response.

In [4]:

prepared_request = requests.Request(
    'GET', 
    'http://localhost:80/headers', 
    headers={}
).prepare()

session = requests.Session()
response = session.send(prepared_request)

print(response.content.decode("utf-8"))

{
  "headers": {
    "Accept-Encoding": "identity", 
    "Host": "localhost", 
    "User-Agent": "python-urllib3/2.5.0"
  }
}



Even though we explicitly specified `headers={}`, it still results in some filled headers.

## Query params

With the `params` argument in methods, you can specify specify the query parameters. `requests` automatically add them to the URL.

---

The following cell displays the complete URL for a `get` request when the `params` argument specified. 

In [11]:
requests.get(
    "http://localhost:80/anything",
    params={"a": 10, "b": "some_value"}
).url

'http://localhost:80/anything?a=10&b=some_value'

## Reponse

The methods that provide requests retun the `requests.Response` instance. The following tale shows the most useful attributes of the `requests.Response` class.

| Attribute / Method       | Description                                                |
|--------------------------|------------------------------------------------------------|
| `status_code`            | Integer HTTP status code (e.g., 200, 404).                 |
| `headers`                | Response headers as a case-insensitive dict.               |
| `text`                   | Body decoded to a string using detected encoding.          |
| `content`                | Raw response body as bytes.                                |
| `json()`                 | Parses the body as JSON and returns a Python object.       |
| `url`                    | Final request URL (after redirects).                       |
| `ok`                     | `True` if status code < 400.                               |
| `reason`                 | Reason phrase from the server (e.g., “Not Found”).         |
| `cookies`                | Cookies provided by the server.                            |
| `elapsed`                | Time taken for the request.                                |
| `history`                | List of intermediate responses (e.g., redirects).          |
| `raise_for_status()`     | Raises an exception for HTTP status codes ≥ 400.           |

---

The follwoing cell sends the request and saves the ouput as `response`.

In [6]:
response = requests.get("http://localhost:80/anything")
type(response)

requests.models.Response

The following cell displays the attribute `url` of the response.

In [7]:
response.url

'http://localhost:80/anything'

### Content

The response has several attributes that represent the content at different stages of processing:

- `content`: the pure butes.
- `text`: content decoded with detected encoding.
- `json`: returns a python dictionary if the response content can be decerialized from JSON.

---

Examine different components of the response as defined in the following cell.

In [18]:
response = requests.get("http://localhost:80/anything")

The following cell displays the `content` attribute.

In [16]:
response.content

b'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Connection": "keep-alive", \n    "Host": "localhost", \n    "User-Agent": "python-requests/2.32.5"\n  }, \n  "json": null, \n  "method": "GET", \n  "origin": "172.17.0.1", \n  "url": "http://localhost/anything"\n}\n'

For the `text` ouput the same, but encoded as a `str` data type.

In [17]:
response.text

'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Connection": "keep-alive", \n    "Host": "localhost", \n    "User-Agent": "python-requests/2.32.5"\n  }, \n  "json": null, \n  "method": "GET", \n  "origin": "172.17.0.1", \n  "url": "http://localhost/anything"\n}\n'

With `json` method you can get the body of the response as python dictionary.

In [9]:
response.json()

{'args': {},
 'data': '',
 'files': {},
 'form': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Connection': 'keep-alive',
  'Host': 'localhost',
  'User-Agent': 'python-requests/2.32.5'},
 'json': None,
 'method': 'GET',
 'origin': '172.17.0.1',
 'url': 'http://localhost/anything'}